Dou et al. (2020) model

Agora em Julia versão 1.5.3

Obs: não consigo aumentar grid para 1000 porque não há memória o suficiente. Vamos uasr grid=100 mesmo

In [60]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using VegaLite
using DataFrames


#packages to increase the speed of the code
using BenchmarkTools
using Profile # @profile command
using ProfileVega #profiling using graphs
# using StatProfilerHTML
using ProfileView #profiling using graphs
# using Traceur #analyses if your function has performance flaws such as changing the type of a variable


Game parameters

In [61]:
const μ = Float64(4.566) #number of months in each period
const ρ = Float64(0.884) # (1 - ρ) is the firm's depreciation rate each period
# const ρ = Float64(0.5) #testing new depreciation rate
const β = Float64(9.84) #inverse of the speed of learning
const c0 = Float64(0.044) #fixed cost of going to court
const c1 = Float64(0.015) #variable cost of going to court


const hs0 = Float64(0.28) #senior's initial skill level
const hj0 = Float64(0.36) #junior's initial skill level

const λj = Float64(0.346)

#const is to alert Julia that these global variables will not change, so she does not have to worry about then

0.346

Data

In [62]:
const Vmax = Float64(1.0) #firm's maximum reorganization value
const L = Float64(0.25) #firm's maximum liquidation value
const Ds = Float64(0.32) #senior's debt face value
const Dj = Float64(0.68) #junior's debt face value

const D = Ds + Dj #total debt, used to scale costs

1.0

Initial definitions: costs, maximum value of the firm, maximum number of periods



In [63]:
#number of periods
function max_turns(Vmax, L, ρ)
    t = 0
    
    #using the specification of the theoretical model
    #we are not considering the change in indexation from t to t+1 yet.
    while(ρ^(t-1) * Vmax > L)
        t = t+1
    end
    return t
end


T = max_turns(Vmax, L, ρ)
T

13

In [64]:
#checking if T is indeed the final period
@show t = T
@show ρ^(t-1) * Vmax
@show L
@show ρ^(t-1) * Vmax > L 


@show t = T-1
@show ρ^(t-1) * Vmax
@show L
@show ρ^(t-1) * Vmax > L




t = T = 13
ρ ^ (t - 1) * Vmax = 0.2277336360006909
L = 0.25
ρ ^ (t - 1) * Vmax > L = false
t = T - 1 = 12
ρ ^ (t - 1) * Vmax = 0.2576172352948992
L = 0.25
ρ ^ (t - 1) * Vmax > L = true


true

### Important: we will create a game with T+1 periods

So we'll be able to include period t==0 in the arrays as index 1

So the notation is: period t is represented by the index t-1


In [65]:
#cost function
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
    end
end

#precisa de um 'end' para o if e um 'end' para a função
    
#test
@show Ct(0)    
@show Ct(1)
@show Ct(2)
@show Ct(2) == c0 * D + c1 * (1) * D


Ct(0) = 0
Ct(1) = 0
Ct(2) = 0.059
Ct(2) == c0 * D + c1 * 1 * D = true


true

In [66]:
#maximum value of reorganization each period
function Vt(Vmax, ρ, t)
    
    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end
    
    
end

@show Vt(Vmax, ρ, 1)
@show Vt(Vmax, ρ, 2)
@show Vt(Vmax, ρ, 3)
@show Vt(Vmax, ρ, T)
@show Vt(Vmax, ρ, T+1)
#index 1 corresponds to t=0 and 2 to t=1...

Vt(Vmax, ρ, 1) = 1.0
Vt(Vmax, ρ, 2) = 1.0
Vt(Vmax, ρ, 3) = 0.884
Vt(Vmax, ρ, T) = 0.2576172352948992
Vt(Vmax, ρ, T + 1) = 0.2277336360006909


0.2277336360006909

In [67]:
#liquidation payoffs

function s_L(t)
    return min(L - Ct(t), Ds)
end

@show s_L(0)
@show s_L(1)
@show s_L(2)


function j_L(t)
    return min(L - Ct(t) - s_L(t), Dj)
end
    
@show j_L(1)

s_L(0) = 0.25
s_L(1) = 0.25
s_L(2) = 0.191
j_L(1) = 0.0


0.0

Skill levels' grid and Probability Mass Function(PMF)

Important: we won't use the skill levels directly in the functions, we will use their indexes. For example, index = 40 corresponds to $\theta_{K,t} = 0.4$.

In [68]:
#grid size
const grid = 100

hlow = 0.01
hhigh = 1.0


hvals = LinRange(hlow, hhigh, grid)

hvals
hvals[6]

0.06

In [69]:
#array to tell the size of the pie at period t, according to reorganization skill levels
#arguments: (period, θkt)
U = Array{Float64}(undef, T+1, grid)


for t in 1:T+1
    for h in 1:grid
        U[t,h] = hvals[h] * Vt(Vmax, ρ, t) - Ct(t)
    end
end

In [70]:
#cumulative distribution function
function cdf(x, lt)
    
    #return the cdf of x given the lower bound lt
    #geq than 1 because of our discretization method 
    if(x >= 1.0)
        
        return 1.0
    
    else
        
        if(x >= lt)
            return 1.0 - ((1.0-x)^β)/((1-lt)^β)
        else
            return 0.0
        end
    end
end

#test      
@show cdf(0.5, 1)

@show cdf(0.5, 0.4)

@show cdf(0.99, 0.01)

@show cdf(1, 0.01)

@show cdf(1, 0.9)

cdf(0.5, 1) = 0.0
cdf(0.5, 0.4) = 0.8337136736331454
cdf(0.99, 0.01) = 1.0
cdf(1, 0.01) = 1.0
cdf(1, 0.9) = 1.0


1.0

In [71]:
#pmf: each point of the grid is at the center of the cdf
function pmf_cdf(lt, δ = hlow/2)
    pmf = zeros(grid)
    
    for (i, h) in enumerate(hvals)
        pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
    end
    
    return pmf
end


lt = 0.01
pmf_cdf(lt)

100-element Array{Float64,1}:
 0.048602089527521986
 0.09088953125821697
 0.08301186884522238
 0.07574579901540057
 0.06904947823155516
 0.06288353216214904
 0.05721093367839869
 0.05199688568380789
 0.04720870863310611
 0.04281573260029048
 0.038789193758253626
 0.03510213513518168
 0.03172931151561542
 ⋮
 3.7337122282821156e-10
 1.615731992643532e-10
 6.408462649432067e-11
 2.2833512858255745e-11
 7.107758825952715e-12
 1.856736986383112e-12
 3.83026943495679e-13
 5.651035195342047e-14
 4.9960036108132044e-15
 2.220446049250313e-16
 0.0
 0.0

In [72]:
#generating the pmfs

#probability mass function
pmf = Array{Float64}(undef,grid,grid)


for (t, θt) in enumerate(hvals)
    pmf[t,:] = pmf_cdf(θt)
end


#we need a pmf for when we have lkt and want to know θk,t+1
#we will call it 'pmf2'
pmf2 = similar(pmf)

for i in eachindex(hvals)
    for j in eachindex(hvals)
        pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
    end
end

#rows are the skill levels today, columns are the skill levels tomorrow
pmf2[1,:] - pmf[1,:]

#should be nice to plot the two pmfs, so we will get the intuition easily

100-element Array{Float64,1}:
 -0.046239926421080726
 -0.08201062544335427
 -0.06652389304197329
 -0.05297653770265348
 -0.04117778693388258
 -0.03095216055945394
 -0.022138473358402036
 -0.014588889342129194
 -0.008168025651978328
 -0.0027521041142639507
  0.0017718514572064026
  0.0055067740688657385
  0.008546270271650315
  ⋮
  7.710965259700306e-9
  3.4895426000488317e-9
  1.4511143376026725e-9
  5.43807346649936e-10
  1.7875565752175066e-10
  4.95698020399729e-11
  1.093125330283789e-11
  1.7426855209729608e-12
  1.6978822815991093e-13
  7.239768851825965e-15
  5.665150746454088e-17
  8.758153619997799e-22

Arrays with the continuation values

In [73]:
#continuation values
#period t (not t+1), θkt, ℓkt, ℓmt
s_W = zeros(T+1, grid, grid, grid)
j_W = similar(s_W)


#optimal payments
#period, θkt, ℓmt, outputs
Pst_array = zeros(T, grid, grid, 6) #we won't need payment proposal for period T, a.k.a. "T+1"
Pjt_array = similar(Pst_array)

13×100×100×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [74]:
#populating the last period with the liquidation values
s_W[T+1, :, :, :] .= s_L(T+1); #we need to use .= because it is a broadcast
j_W[T+1, :, :, :] .= j_L(T+1);

In [75]:
#cutoff function: upgrade

function getcutoff_m(t, pkt, hkt, m_W)

    #we extract the diagonal values, where θmt == lmt, to search for the cutoff
    diag_vals = @views diag(m_W[t+1, :, :, hkt])
    
    
    #we don't need to worry about negative payment offers because in this case the cutoff will be 1
    #if the pkt is greater than all the continuation values of the adversary, then it will return '101' (grid+1)
    return searchsortedfirst(diag_vals, pkt)
end

#should return 100 because J's continuation value at T is zero
@btime getcutoff_m(T, 0.01, 50, j_W)


  227.200 ns (1 allocation: 896 bytes)


101

In [76]:
#function to "filter" the cutoff

function cutoff_m(t, pkt, hkt, lmt, m_W)
    #for now, it will just tell us if the payment offer is negative
    
    #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
    if(pkt<0)
         return grid+2
    end
    
    cmt = getcutoff_m(t, pkt, hkt, m_W)
    
    return cmt

    
#     #if there is no cutoff (cutoff==101), then cutoff_m will return 101
#     if(cmt==101)
#         return cmt
#     else
#         return max(cmt, lmt)
#     end
end

@show cutoff_m(T, -0.01, 50, 40, j_W)
@show cutoff_m(T, 0.01, 50, 40, j_W)
@btime cutoff_m(T, 0.01, 50, 40, j_W)

cutoff_m(T, -0.01, 50, 40, j_W) = 102
cutoff_m(T, 0.01, 50, 40, j_W) = 101
  228.452 ns (1 allocation: 896 bytes)


101

In [77]:
#function to select the pmf function as it is writen in the paper

function h_m(t, lmt, pmf, pmf2)
    #takes the indexes of cutoff and lmt
        
    #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
    if(t <= 1)
        return pmf[lmt, 1:end]
    else
        return pmf2[lmt, 1:end]
    end
end
    
#test
@btime h_m(2,50, pmf, pmf2)

  152.280 ns (1 allocation: 896 bytes)


100-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 4.625716199215591e-6
 2.128501871030038e-6
 8.99899204442307e-7
 3.428623147305904e-7
 1.1459581460290833e-7
 3.2321165203169804e-8
 7.253369111549412e-9
 1.1778647340830674e-9
 1.1710929608790348e-10
 5.110233142412765e-12
 4.3007226426965164e-14
 1.7849057398291997e-18

In [78]:
#version passing global variables as parameters


grid_payment = 1000
share = LinRange(hlow, hhigh-hlow, grid_payment)

#testing cutoff in this newer version of Pkt

#function to calculate the optimal payment offer and directly populate the array

function Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)
    
    
    
    Pkt_grid = Vector{Float64}(undef, length(share)+1)

#     Pkt_grid = zeros(length(share)+1)
    
    #the first input is -Vmax, to assure that m will reject all offers. It is the waiting payoff
    Pkt_grid[1] = -Vmax

    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] .= U[t, hkt] .* share
    
    #discarding values below the lowest continuation value of the opponent
    #this way we won't mistake a waiting offer for a reorganization offer

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
#     lowest = m_W[t+1, lmt, lmt, hkt]

#     Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])
    
    #screening cutoffs
    cmt = Vector{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cmt
    for (i, Pkt) in enumerate(Pkt_grid)
        cmt[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    
    payoff = zeros(length(Pkt_grid))
    
    
    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    probm = h_m(t,lmt, pmf, pmf2)
    
    for (i, pkt) in enumerate(Pkt_grid)
        if(cmt[i]==grid+2) 
            #means that pkt<0, so it will be always rejected
            #lmt+1 == lmt
            payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) 
        
        
        
        elseif(cmt[i] == grid+1) 
            #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

            #since sum(probm[lmt:end])==1, we didn't include it here
            payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end]))
        
        else
            #cases where cmt[i] is between 1 and grid
            
            if(cmt[i] < lmt)
                #means that pkt<minimum continuation value of m, so it will be always rejected
                #similar to pkt<0
                #lmt+1 == lmt
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end]))
                
            elseif(cmt[i]==lmt)
                
                #accepted only if hmt+1==lmt
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * probm[lmt]
                
                #rejected otherwise
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(lmt+1):end]))
                
            else
                #cmt[i] > lmt
                #accepted if hmt+1 <=cmt[i]
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * sum(@view(probm[lmt:cmt[i]]))
                
                
                #rejected otherwise
                #lmt+1 = cmt[i]
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, cmt[i]]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(cmt[i]+1):end]))
            
            end
        end
    end
    
    
    
    #payoffs matrix####

#     payoff_reorg, index_reorg = findmax(payoff[2:end])
    payoff_reorg, index_reorg = findmax(payoff)
    payment = Pkt_grid[index_reorg]

    

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))
    
    
    #condition to always show the payoff reorg
#     if(policy==3.0)
#         payment = Pkt_grid[index_reorg+1]
#     else
#         payment = -Vmax
#     end

    #populates the array
    Pkt_array[t, hkt, lmt, :] .= payment, cmt[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end




#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U = T, 100, 60, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)
@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

#testing hk = 0.5
hkt = 50
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)


Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U) = [0.0003361723529489924, 101.0, 0.03328106294195025, 0.01100000000000001, 0.03328106294195025, 3.0]
  252.601 μs (2015 allocations: 994.13 KiB)
Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U) = [-1.0, 102.0, 0.011000000000000017, 0.011000000000000017, 0.026000000000000023, 1.0]
  51.701 μs (15 allocations: 25.38 KiB)


6-element view(::Array{Float64,4}, 13, 50, 60, :) with eltype Float64:
  -1.0
 102.0
   0.011000000000000017
   0.011000000000000017
   0.026000000000000023
   1.0

In [79]:
typeof(Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U))

SubArray{Float64,1,Array{Float64,4},Tuple{Int64,Int64,Int64,Base.Slice{Base.OneTo{Int64}}},true}

In [80]:
@code_warntype Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)
#show, código todo azul!!!! Linhas amareladas não fazem mal

Variables
  #self#::Core.Compiler.Const(Pkt, false)
  t::Int64
  hkt::Int64
  lmt::Int64
  k_W::Array{Float64,4}
  m_W::Array{Float64,4}
  k_L::Core.Compiler.Const(s_L, false)
  Pkt_array::Array{Float64,4}
  pmf::Array{Float64,2}
  pmf2::Array{Float64,2}
  share::LinRange{Float64}
  U::Array{Float64,2}
  @_13::Int64
  @_14::Int64
  Pkt_grid::Array{Float64,1}
  cmt::Array{Int64,1}
  @_17::Union{Nothing, Tuple{Tuple{Int64,Float64},Tuple{Int64,Int64}}}
  payoff::Array{Float64,1}
  probm::Array{Float64,1}
  @_20::Union{Nothing, Tuple{Tuple{Int64,Float64},Tuple{Int64,Int64}}}
  payoff_reorg::Float64
  index_reorg::Int64
  payment::Float64
  payoff_liq::Float64
  payoff_wait::Float64
  payoff_max::Float64
  policy::Int64
  i@_28::Int64
  Pkt::Float64
  @_30::Int64
  pkt::Float64
  @_32::Int64
  i@_33::Int64
  @_34::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,UnitRange{Int64},Int64,Int64},true}
  @_35::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
  @_36::SubArra

7 ──        Core.Compiler.Const(:(@_34 = false), false)
8 ┄─ %77  = @_34::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,UnitRange{Int64},Int64,Int64},true}
│           true
│    %79  = (lastindex)(pmf, 2)::Int64
│    %80  = (hkt:%79)::UnitRange{Int64}
│           (@_35 = (view)(pmf, hkt, %80))
└───        goto #10
9 ──        Core.Compiler.Const(:(@_35 = false), false)
10 ┄ %84  = @_35::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
│    %85  = Main.dot(%77, %84)::Float64
│    %86  = (%69 + %85)::Float64
│           Base.setindex!(payoff, %86, i@_33)
└───        goto #48
11 ─ %89  = Base.getindex(cmt, i@_33)::Int64
│    %90  = (Main.grid + 1)::Core.Compiler.Const(101, false)
│    %91  = (%89 == %90)::Bool
└───        goto #17 if not %91
12 ─ %93  = Base.getindex(payoff, i@_33)::Float64
│           true
│    %95  = (lastindex)(U, 2)::Int64
│    %96  = (hkt:%95)::UnitRange{Int64}
│           (@_36 = (view)(U, t, %96))
└───        goto #14
13 ─        Core.Compi

38 ─        Core.Compiler.Const(:(@_46 = false), false)
39 ┄ %209 = @_46::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
│    %210 = Main.dot(%202, %209)::Float64
│           true
│    %212 = probm::Array{Float64,1}
│    %213 = Base.getindex(cmt, i@_33)::Int64
│    %214 = (lmt:%213)::UnitRange{Int64}
│           (@_47 = (view)(%212, %214))
└───        goto #41
40 ─        Core.Compiler.Const(:(@_47 = false), false)
41 ┄ %218 = @_47::SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}
│    %219 = Main.sum(%218)::Float64
│    %220 = (%210 * %219)::Float64
│    %221 = (%193 + %220)::Float64
│           Base.setindex!(payoff, %221, i@_33)
│    %223 = Base.getindex(payoff, i@_33)::Float64
│           true
│    %225 = (t + 1)::Int64
│    %226 = (lastindex)(k_W, 2)::Int64
│    %227 = (hkt:%226)::UnitRange{Int64}
│    %228 = Base.getindex(cmt, i@_33)::Int64
│           (@_48 = (view)(k_W, %225, %227, hkt, %228))
└───        goto #43
42 ─        Core.Compi

In [81]:
# Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

function Pst(t, θst, ℓjt)
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U)
end


function Pjt(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U)
end

Pjt (generic function with 1 method)

In [82]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T
t = T
for h in 1:grid
    for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
    end
end

### Começo dos testes com argumentos posicionais

In [185]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T


# Supondo que eu passe todos os argumentos em todas as funções
t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U = T, 100, 60, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U


t = T
@time begin
    for h in 1:grid
        for l in 1:grid
                Pkt(t, h, l, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)
        end
    end
end


  1.533967 seconds (1.99 M allocations: 1.459 GiB, 11.23% gc time)


In [187]:
@code_warntype Pkt(t, 1, 1, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

Variables
  #self#::Core.Compiler.Const(Pkt, false)
  t::Int64
  hkt::Int64
  lmt::Int64
  k_W::Array{Float64,4}
  m_W::Array{Float64,4}
  k_L::Core.Compiler.Const(j_L, false)
  Pkt_array::Array{Float64,4}
  pmf::Array{Float64,2}
  pmf2::Array{Float64,2}
  share::LinRange{Float64}
  U::Array{Float64,2}
  @_13::Int64
  @_14::Int64
  Pkt_grid::Array{Float64,1}
  cmt::Array{Int64,1}
  @_17::Union{Nothing, Tuple{Tuple{Int64,Float64},Tuple{Int64,Int64}}}
  payoff::Array{Float64,1}
  probm::Array{Float64,1}
  @_20::Union{Nothing, Tuple{Tuple{Int64,Float64},Tuple{Int64,Int64}}}
  payoff_reorg::Float64
  index_reorg::Int64
  payment::Float64
  payoff_liq::Float64
  payoff_wait::Float64
  payoff_max::Float64
  policy::Int64
  i@_28::Int64
  Pkt::Float64
  @_30::Int64
  pkt::Float64
  @_32::Int64
  i@_33::Int64
  @_34::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,UnitRange{Int64},Int64,Int64},true}
  @_35::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
  @_36::SubArra

│           (@_34 = (view)(k_W, %71, %73, hkt, lmt))
└───        goto #8
7 ──        Core.Compiler.Const(:(@_34 = false), false)
8 ┄─ %77  = @_34::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,UnitRange{Int64},Int64,Int64},true}
│           true
│    %79  = (lastindex)(pmf, 2)::Int64
│    %80  = (hkt:%79)::UnitRange{Int64}
│           (@_35 = (view)(pmf, hkt, %80))
└───        goto #10
9 ──        Core.Compiler.Const(:(@_35 = false), false)
10 ┄ %84  = @_35::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
│    %85  = Main.dot(%77, %84)::Float64
│    %86  = (%69 + %85)::Float64
│           Base.setindex!(payoff, %86, i@_33)
└───        goto #48
11 ─ %89  = Base.getindex(cmt, i@_33)::Int64
│    %90  = (Main.grid + 1)::Core.Compiler.Const(101, false)
│    %91  = (%89 == %90)::Bool
└───        goto #17 if not %91
12 ─ %93  = Base.getindex(payoff, i@_33)::Float64
│           true
│    %95  = (lastindex)(U, 2)::Int64
│    %96  = (hkt:%95)::UnitRange{Int64}
│        

│           (@_46 = (view)(pmf, hkt, %205))
└───        goto #39
38 ─        Core.Compiler.Const(:(@_46 = false), false)
39 ┄ %209 = @_46::SubArray{Float64,1,Array{Float64,2},Tuple{Int64,UnitRange{Int64}},true}
│    %210 = Main.dot(%202, %209)::Float64
│           true
│    %212 = probm::Array{Float64,1}
│    %213 = Base.getindex(cmt, i@_33)::Int64
│    %214 = (lmt:%213)::UnitRange{Int64}
│           (@_47 = (view)(%212, %214))
└───        goto #41
40 ─        Core.Compiler.Const(:(@_47 = false), false)
41 ┄ %218 = @_47::SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}
│    %219 = Main.sum(%218)::Float64
│    %220 = (%210 * %219)::Float64
│    %221 = (%193 + %220)::Float64
│           Base.setindex!(payoff, %221, i@_33)
│    %223 = Base.getindex(payoff, i@_33)::Float64
│           true
│    %225 = (t + 1)::Int64
│    %226 = (lastindex)(k_W, 2)::Int64
│    %227 = (hkt:%226)::UnitRange{Int64}
│    %228 = Base.getindex(cmt, i@_33)::Int64
│           (@_48 = (view)(k_W, %2

In [182]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T


# BENCHMARK

t = T
@time begin
    for h in 1:grid
        for l in 1:grid
                Pjt(t, h, l)
        end
    end
end


  1.503081 seconds (1.99 M allocations: 1.459 GiB, 11.26% gc time)


In [84]:
@code_warntype Pjt(T,1,1)

Variables
  #self#::Core.Compiler.Const(Pjt, false)
  t::Int64
  θjt::Int64
  ℓst::Int64

Body::Any
1 ─ %1 = Main.Pkt(t, θjt, ℓst, Main.j_W, Main.s_W, Main.j_L, Main.Pjt_array, Main.pmf, Main.pmf2, Main.share, Main.U)::Any
└──      return %1


ok, então ela não reconhece os tipos dos parâmetros de Pkt


vamos tentar reescrever Pkt, definindo já os tipos dos parâmetros

In [85]:
#version passing global variables as parameters


grid_payment = 1000
share = LinRange(hlow, hhigh-hlow, grid_payment)

#testing cutoff in this newer version of Pkt

#function to calculate the optimal payment offer and directly populate the array

function Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,Int64,Int64,Base.Slice{Base.OneTo{Int64}}},true}
    
    
    
    Pkt_grid = Vector{Float64}(undef, length(share)+1)

#     Pkt_grid = zeros(length(share)+1)
    
    #the first input is -Vmax, to assure that m will reject all offers. It is the waiting payoff
    Pkt_grid[1] = -Vmax

    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] .= U[t, hkt] .* share
    
    #discarding values below the lowest continuation value of the opponent
    #this way we won't mistake a waiting offer for a reorganization offer

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
#     lowest = m_W[t+1, lmt, lmt, hkt]

#     Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])
    
    #screening cutoffs
    cmt = Vector{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cmt
    for (i, Pkt) in enumerate(Pkt_grid)
        cmt[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    
    payoff = zeros(length(Pkt_grid))
    
    
    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    probm = h_m(t,lmt, pmf, pmf2)
    
    for (i, pkt) in enumerate(Pkt_grid)
        if(cmt[i]==grid+2) 
            #means that pkt<0, so it will be always rejected
            #lmt+1 == lmt
            payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) 
        
        
        
        elseif(cmt[i] == grid+1) 
            #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

            #since sum(probm[lmt:end])==1, we didn't include it here
            payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end]))
        
        else
            #cases where cmt[i] is between 1 and grid
            
            if(cmt[i] < lmt)
                #means that pkt<minimum continuation value of m, so it will be always rejected
                #similar to pkt<0
                #lmt+1 == lmt
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end]))
                
            elseif(cmt[i]==lmt)
                
                #accepted only if hmt+1==lmt
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * probm[lmt]
                
                #rejected otherwise
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(lmt+1):end]))
                
            else
                #cmt[i] > lmt
                #accepted if hmt+1 <=cmt[i]
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * sum(@view(probm[lmt:cmt[i]]))
                
                
                #rejected otherwise
                #lmt+1 = cmt[i]
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, cmt[i]]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(cmt[i]+1):end]))
            
            end
        end
    end
    
    
    
    #payoffs matrix####

#     payoff_reorg, index_reorg = findmax(payoff[2:end])
    payoff_reorg, index_reorg = findmax(payoff)
    payment = Pkt_grid[index_reorg]

    

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))
    
    
    #condition to always show the payoff reorg
#     if(policy==3.0)
#         payment = Pkt_grid[index_reorg+1]
#     else
#         payment = -Vmax
#     end

    #populates the array
    Pkt_array[t, hkt, lmt, :] .= payment, cmt[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end




#testing hk = 1
t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U = T, 100, 60, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)
@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

#testing hk = 0.5
hkt = 50
@show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

@btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)


Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U) = [0.0003361723529489924, 101.0, 0.03328106294195025, 0.01100000000000001, 0.03328106294195025, 3.0]
  253.599 μs (2015 allocations: 994.13 KiB)
Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U) = [-1.0, 102.0, 0.011000000000000017, 0.011000000000000017, 0.026000000000000023, 1.0]
  51.600 μs (15 allocations: 25.38 KiB)


6-element view(::Array{Float64,4}, 13, 50, 60, :) with eltype Float64:
  -1.0
 102.0
   0.011000000000000017
   0.011000000000000017
   0.026000000000000023
   1.0

In [86]:
function Pjt2(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U)
end

Pjt2 (generic function with 1 method)

In [181]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T


# Argumentos opcionais

t = T
@time begin
    for h in 1:grid
        for l in 1:grid
                Pjt2(t, h, l)
        end
    end
end


  1.639185 seconds (1.99 M allocations: 1.459 GiB, 12.85% gc time)


In [88]:
@code_warntype Pjt2(T,1, 1)

Variables
  #self#::Core.Compiler.Const(Pjt2, false)
  t::Int64
  θjt::Int64
  ℓst::Int64

Body::Any
1 ─ %1 = Main.Pkt(t, θjt, ℓst, Main.j_W, Main.s_W, Main.j_L, Main.Pjt_array, Main.pmf, Main.pmf2, Main.share, Main.U)::Any
└──      return %1


Não adianta só especificar o tipo do output

Tentativa 2: espeficiar os tipos dos argumentos de Pkt, e depois definir Pjt3

In [89]:
#version passing global variables as parameters


grid_payment = 1000
share = LinRange(hlow, hhigh-hlow, grid_payment)

#testing cutoff in this newer version of Pkt

#function to calculate the optimal payment offer and directly populate the array

function Pkt(t::Int64, 
        hkt::Int64,
        lmt::Int64, 
        k_W=j_W::Array{Float64,4}, 
        m_W=s_W, 
        k_L=j_L, Pkt_array=Pjt_array, pmf=pmf, pmf2=pmf2, share=share, U=U)
    
    
    Pkt_grid = Vector{Float64}(undef, length(share)+1)

#     Pkt_grid = zeros(length(share)+1)
    
    #the first input is -Vmax, to assure that m will reject all offers. It is the waiting payoff
    Pkt_grid[1] = -Vmax

    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] .= U[t, hkt] .* share
    
    #discarding values below the lowest continuation value of the opponent
    #this way we won't mistake a waiting offer for a reorganization offer

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
#     lowest = m_W[t+1, lmt, lmt, hkt]

#     Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])
    
    #screening cutoffs
    cmt = Vector{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cmt
    for (i, Pkt) in enumerate(Pkt_grid)
        cmt[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    
    payoff = zeros(length(Pkt_grid))
    
    
    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    probm = h_m(t,lmt, pmf, pmf2)
    
    for (i, pkt) in enumerate(Pkt_grid)
        if(cmt[i]==grid+2) 
            #means that pkt<0, so it will be always rejected
            #lmt+1 == lmt
            payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) 
        
        
        
        elseif(cmt[i] == grid+1) 
            #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

            #since sum(probm[lmt:end])==1, we didn't include it here
            payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end]))
        
        else
            #cases where cmt[i] is between 1 and grid
            
            if(cmt[i] < lmt)
                #means that pkt<minimum continuation value of m, so it will be always rejected
                #similar to pkt<0
                #lmt+1 == lmt
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end]))
                
            elseif(cmt[i]==lmt)
                
                #accepted only if hmt+1==lmt
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * probm[lmt]
                
                #rejected otherwise
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(lmt+1):end]))
                
            else
                #cmt[i] > lmt
                #accepted if hmt+1 <=cmt[i]
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * sum(@view(probm[lmt:cmt[i]]))
                
                
                #rejected otherwise
                #lmt+1 = cmt[i]
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, cmt[i]]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(cmt[i]+1):end]))
            
            end
        end
    end
    
    
    
    #payoffs matrix####

#     payoff_reorg, index_reorg = findmax(payoff[2:end])
    payoff_reorg, index_reorg = findmax(payoff)
    payment = Pkt_grid[index_reorg]

    

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))
    
    
    #condition to always show the payoff reorg
#     if(policy==3.0)
#         payment = Pkt_grid[index_reorg+1]
#     else
#         payment = -Vmax
#     end

    #populates the array
    Pkt_array[t, hkt, lmt, :] .= payment, cmt[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end


#padrões definidos como se ela fosse j

#testing hk = 1
t, hkt, lmt = T, 100, 60
@show Pkt(t, hkt, lmt)
@btime Pkt(t, hkt, lmt)

# #testing hk = 0.5
# hkt = 50
# @show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

# @btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)


Pkt(t, hkt, lmt) = [0.011021001733165616, 101.0, 0.022596233561733625, 0.0, 0.022596233561733625, 3.0]
  248.700 μs (1692 allocations: 963.80 KiB)


6-element view(::Array{Float64,4}, 13, 100, 60, :) with eltype Float64:
   0.011021001733165616
 101.0
   0.022596233561733625
   0.0
   0.022596233561733625
   3.0

In [90]:
typeof(t)

Int64

In [91]:
@code_warntype Pkt(t, hkt, lmt)

Variables
  #self#::Core.Compiler.Const(Pkt, false)
  t::Int64
  hkt::Int64
  lmt::Int64

Body::Any
1 ─ %1 = Core.apply_type(Main.Array, Main.Float64, 4)::Core.Compiler.Const(Array{Float64,4}, false)
│   %2 = Core.typeassert(Main.j_W, %1)::Array{Float64,4}
│   %3 = (#self#)(t, hkt, lmt, %2)::Any
└──      return %3


In [92]:
using Parameters

In [165]:
typeof(share)

LinRange{Float64}

In [174]:
@with_kw struct param12345678
    k_W::Array{Float64,4}=j_W
    m_W::Array{Float64,4}=s_W
#     k_L=j_L
    Pkt_array::Array{Float64,4}=Pjt_array
    pmf::Array{Float64,2}=pmf
    pmf2::Array{Float64,2}=pmf2
    share::LinRange{Float64}=share
    U::Array{Float64,2}=U
end


    

param12345678

In [175]:
jparam = param12345678();

In [177]:
#version passing global variables as parameters


grid_payment = 1000
share = LinRange(hlow, hhigh-hlow, grid_payment)

#testing cutoff in this newer version of Pkt

#function to calculate the optimal payment offer and directly populate the array

function Pkt(param, t, hkt, lmt, k_L=j_L)
    
    @unpack k_W, m_W, Pkt_array, pmf, pmf2, share, U = param
    
    
    Pkt_grid = Vector{Float64}(undef, length(share)+1)

#     Pkt_grid = zeros(length(share)+1)
    
    #the first input is -Vmax, to assure that m will reject all offers. It is the waiting payoff
    Pkt_grid[1] = -Vmax

    
    #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
    #the reorganization payoff is U_{t} (\theta_{t+1})
    Pkt_grid[2:end] .= U[t, hkt] .* share
    
    #discarding values below the lowest continuation value of the opponent
    #this way we won't mistake a waiting offer for a reorganization offer

    #descartando os valores que são menores que o menor valor de continuação possível do adversário
    #assim não confundiremos uma proposta de reorganização com uma de espera
#     lowest = m_W[t+1, lmt, lmt, hkt]

#     Pkt_grid[2:end] = @. ifelse(Pkt_grid[2:end] < lowest, lowest, Pkt_grid[2:end])
    
    #screening cutoffs
    cmt = Vector{Int64}(undef, length(Pkt_grid))

     #loop for calculating the cmt
    for (i, Pkt) in enumerate(Pkt_grid)
        cmt[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
    end

    
    payoff = zeros(length(Pkt_grid))
    
    
    #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
    probm = h_m(t,lmt, pmf, pmf2)
    
    for (i, pkt) in enumerate(Pkt_grid)
        if(cmt[i]==grid+2) 
            #means that pkt<0, so it will be always rejected
            #lmt+1 == lmt
            payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) 
        
        
        
        elseif(cmt[i] == grid+1) 
            #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

            #since sum(probm[lmt:end])==1, we didn't include it here
            payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end]))
        
        else
            #cases where cmt[i] is between 1 and grid
            
            if(cmt[i] < lmt)
                #means that pkt<minimum continuation value of m, so it will be always rejected
                #similar to pkt<0
                #lmt+1 == lmt
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end]))
                
            elseif(cmt[i]==lmt)
                
                #accepted only if hmt+1==lmt
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * probm[lmt]
                
                #rejected otherwise
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(lmt+1):end]))
                
            else
                #cmt[i] > lmt
                #accepted if hmt+1 <=cmt[i]
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * sum(@view(probm[lmt:cmt[i]]))
                
                
                #rejected otherwise
                #lmt+1 = cmt[i]
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, cmt[i]]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(cmt[i]+1):end]))
            
            end
        end
    end
    
    
    
    #payoffs matrix####

#     payoff_reorg, index_reorg = findmax(payoff[2:end])
    payoff_reorg, index_reorg = findmax(payoff)
    payment = Pkt_grid[index_reorg]

    

    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)

    #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
    payoff_wait = payoff[1]

    payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))
    
    
    #condition to always show the payoff reorg
#     if(policy==3.0)
#         payment = Pkt_grid[index_reorg+1]
#     else
#         payment = -Vmax
#     end

    #populates the array
    Pkt_array[t, hkt, lmt, :] .= payment, cmt[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy
    
end


#padrões definidos como se ela fosse j

#testing hk = 1
t, hkt, lmt, param = T, 100, 60, jparam
@show Pkt(param,t, hkt, lmt)
@btime Pkt(param,t, hkt, lmt)


# #testing hk = 0.5
# hkt = 50
# @show Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)

# @btime Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)


Pkt(param, t, hkt, lmt) = [0.011021001733165616, 101.0, 0.022596233561733625, 0.0, 0.022596233561733625, 3.0]
  246.100 μs (1691 allocations: 963.75 KiB)


6-element view(::Array{Float64,4}, 13, 100, 60, :) with eltype Float64:
   0.011021001733165616
 101.0
   0.022596233561733625
   0.0
   0.022596233561733625
   3.0

In [178]:
@code_warntype Pkt(param, t, hkt, lmt)

Variables
  #self#::Core.Compiler.Const(Pkt, false)
  param::param12345678
  t::Int64
  hkt::Int64
  lmt::Int64

Body::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,Int64,Int64,Base.Slice{Base.OneTo{Int64}}},true}
1 ─ %1 = (#self#)(param, t, hkt, lmt, Main.j_L)::SubArray{Float64,1,Array{Float64,4},Tuple{Int64,Int64,Int64,Base.Slice{Base.OneTo{Int64}}},true}
└──      return %1


In [180]:
#populating the arrays to continue the tests
#we populate period T-1, whose index is T


# Argumentos opcionais

t = T
@time begin
    for h in 1:grid
        for l in 1:grid
                Pkt(param, t, h, l)
        end
    end
end


  2.981531 seconds (1.99 M allocations: 1.459 GiB, 6.46% gc time)


### Fim dos testes com argumentos posicionais

In [ ]:
#optimal proposal
function propose(t, hkt, lkt, lmt, Pkt_array)
    
    
    #it just searches for the corresponding Pkt
    payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]
    
    return payoff_max, policy
end

#test with hk = 0.5 (index 50)
t, hkt, lkt, lmt, Pkt_array = T, 50, 1, 60, Pst_array
@show propose(t, hkt, lkt, lmt, Pkt_array)

#test with hk = 1.0 (index 100)
hkt = 100
@show propose(t, hkt, lkt, lmt, Pkt_array)

@btime propose(t, hkt, lkt, lmt, Pkt_array)

In [ ]:
#"cdf" for the discretized distribution
function cdf_discrete(x, lt, pmf)

    #returns the probability that θt takes a value less than or equal to x, given lt
    #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
    return sum(@view(pmf[lt, lt:x]))
end

#test
# @show cdf(0.5, 0.01)

@show cdf_discrete(50, 1, pmf)

@btime cdf_discrete(50,1, pmf)

# @code_warntype cdf_discrete(50,1)

In [ ]:
#function to calculate the payment offer threshold ϕmt

function threshold_m(t, lkt, lmt, Pmt_array)
    
    #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
    #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
    #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
    ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)
    
    
    #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
    @assert ts < 101
    
    return ts
    
end

#test 
t, lkt,lmt, Pmt_array = T, 50, 51, Pst_array
@btime threshold_m(t, lkt,lmt,  Pmt_array)
# @code_warntype threshold_m(t, lkt,lmt,  Pmt_array)

In [ ]:
function prob_liq_m(t, ts, lmt, pmf)
    
    #probability that m will propose liquidation next period
    
    #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
    if(t<=1)
        if(lmt >= ts)
            return 0.0
        else
            return 1.0
        end


    else#if t>0(index>1)

        if(lmt >= ts)
            return 0.0
        else
            return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
        end

    end
    
    
end

t, lkt,lmt, Pmt_array, pmf = T, 50, 51, Pst_array, pmf
ts = threshold_m(t, lkt,lmt,  Pmt_array)
@btime prob_liq_m(t, ts, lmt, pmf)

# @code_warntype prob_liq_m(t, ts, lmt, pmf)
    

In [ ]:
#expected payoff of answering a payment proposal

function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf)
    
    
    #the minimum level of hmt necessary for m to offer a payment
    #it will be used as hmt
    ts = threshold_m(t, lkt, lmt, Pmt_array)
    
    #updated lower bound
    lmt = max(ts, lmt)


    #the possible values of payment given by the opponent
    Pmt_grid = Pmt_array[t, lmt:end, lkt, 1]
    
    
    #the screening cutoffs associated with each payment proposal
    #indexet by k because now it is k who is being screened
    ckt = Vector{Int64}(undef, length(Pmt_grid))
    ckt .= Pmt_array[t, lmt:end, lkt, 2]

    payoff = zero(0.0)
#     payoff = Array{Float64}(undef, length(Pmt_grid))



    for (i, pmt) in enumerate(Pmt_grid)
        
        if(ckt[i]==grid+2)
            #means that the payment offer is negative (waiting offer)
            #the proposal will be rejected for sure, without lowerbound update
            payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]
            
        
        
        elseif(ckt[i]==grid+1)
            #means that there is no cutoff, K will accept the payment offer for all θkt+1
#             payoff += pmt * sum(@view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]
            payoff += pmt * pmf[lmt, lmt+(i-1)]
            
        else
            
            if(ckt[i] < lkt)
                #the proposal will be rejected for sure, without lowerbound update
                payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]
            
            elseif(ckt[i]==lkt)
                #will accept only if hkt+1==lkt
                payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt+(i-1)]
                
                #rejects otherwise
                payoff += dot(@view(k_W[t+1, lkt+1:end, lkt, lmt+(i-1)]), @view(pmf[hkt, lkt+1:end])) * pmf[lmt, lmt+(i-1)]
                
            else
            #now ckt > lkt
                
                if(ckt[i] < hkt)
                    #he rejects, but there is an update in the lowerbound lkt+1==ckt
                    payoff += dot(@view(k_W[t+1, hkt:end, ckt[i], lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]
                
                    
                else
                    
                    #receives pmt if his skill level is below or equal to the cutoff
                    payoff += pmt * sum(@view(pmf[hkt, hkt:ckt[i]])) * pmf[lmt, lmt+(i-1)]
                    
                    
                    #receives continuation value k_W if his skill level greather than the cutoff
                    payoff += dot(@view(k_W[t+1, ckt[i]+1:end , ckt[i], lmt+(i-1)]), @view(pmf[hkt, ckt[i]+1:end])) * pmf[lmt, lmt+(i-1)]
                    
                    
                end
            end
        end
    end
    
    return payoff
end
        
        

#test1: expected payoff of S
t, hkt, lkt, lmt, Pmt_array, k_W, pmf = T, 50, 50, 50, Pjt_array, s_W, pmf
@btime respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf)




In [ ]:
@code_warntype respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf)

In [ ]:
#expected payoff of answering a liquidation proposal

function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
    
    #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
    #U_{t} (\theta_{t+1}) because the skill level is of the next period
    #dot is matrix multiplication
    payoff_reorg = dot(@view(pmf[hkt, hkt:end]), (@view(U[t, hkt:end]) .- m_L(t)))
    
    payoff_liq = k_L(t)
    
    return max(payoff_liq, payoff_reorg)
end


#what happens if J proposes to liquidation at the penultimate period and S answers it?
#hkt = 0.5
t, hkt, lkt, lmt, k_L, m_L, pmf, U = T, 50, 50, 50, s_L, j_L, pmf, U
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
@show dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))
@show k_L(t)

#changing just hkt
#hkt = 1.0
hkt = 100
@show respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
@show dot(pmf[hkt, hkt:end], (U[t, hkt:end] .- m_L(t)))
@show k_L(t)


@btime respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)

In [ ]:
# @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

In [ ]:
#expected payoff of being called to respond
function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U)
    
    probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf)
    
    
    return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
end

#test
t, hkt, lkt, lmt, k_L, m_L, Pmt_array, k_W, pmf, U = T, 50, 50, 50, s_L, j_L, Pst_array, j_W, pmf, U

@show respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U)

@btime respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U)

In [ ]:
#defining S functions based on the generic functions

#Pkt(t, hkt, lmt, k_W, m_W, k_L)
function Pst(t, θst, ℓjt)
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U)
end

#propose(t, hkt, lkt, lmt, Pkt_array)
function s_propose(t, θst, ℓst, ℓjt)
#     r = propose(t, θst, ℓst, ℓjt, Pst_array)
    
#     return r
    
    return propose(t, θst, ℓst, ℓjt, Pst_array)
end

#threshold_m(t, lkt, lmt, Pmt_array)
function threshold_j(t, ℓst, ℓjt)
    return threshold_m(t, ℓst, ℓjt, Pjt_array)
end


#respond_reorg(t, hkt, lkt, lmt, Pmt_array, k_W)
function s_respond_payment(t, θst, ℓst, ℓjt)
    return respond_payment(t, θst, ℓst, ℓjt, Pjt_array, s_W, pmf)
end


#respond_liq(t, hkt, lkt, lmt, k_L, m_L)
function s_respond_liq(t, θst, ℓst, ℓjt)
    return respond_liq(t, θst, ℓst, ℓjt, s_L, j_L, pmf, U)
end

    
#respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L) 
function s_respond(t, θst, ℓst, ℓjt)
    return respond(t, θst, ℓst, ℓjt, Pjt_array, s_W, s_L, j_L, pmf, U)
end

In [ ]:
#J's functions based on the generic functions
function Pjt(t, θjt, ℓst)
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U)
end


function j_propose(t, θjt, ℓjt, ℓst)
    return propose(t, θjt, ℓjt, ℓst, Pjt_array)
end

function threshold_s(t, ℓjt, ℓst)
    return threshold_m(t, ℓjt, ℓst, Pst_array)
end

function j_respond_payment(t, θjt, ℓjt, ℓst)
    return respond_payment(t, θjt, ℓjt, ℓst, Pst_array, j_W, pmf)
end

function j_respond_liq(t, θjt, ℓjt, ℓst)
    return respond_liq(t, θjt, ℓjt, ℓst, j_L, s_L, pmf, U)
end    

function j_respond(t, θjt, ℓjt, ℓst)
    return respond(t, θjt, ℓjt, ℓst, Pst_array, j_W, j_L, s_L, pmf, U)
end

### Populating the game's arrays

#### 1,2,3 testing

In [ ]:
@time begin
    
    t = T
    
    for h in 1:grid
        for l in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
        end
    end
    
end

In [ ]:
#testing functions
# t, hkt, lkt, lmt = T-1, 0.6, 0.4, 0.5
t, hkt, lkt, lmt = T, 60, 40, 50

In [ ]:
@btime s_propose(t, hkt, lkt, lmt)

In [ ]:
@btime threshold_j(t, lkt, lmt)

In [ ]:
#note that threshold_j < threshold_s because j_L(t) < s_L(t) in this case
@btime threshold_s(t, lkt, lmt)

In [ ]:
@btime s_respond_payment(t, hkt, lkt, lmt)

In [ ]:
@btime s_respond_liq(t, hkt, lkt, lmt)

In [ ]:
@btime s_respond(t, hkt, lkt, lmt)

In [ ]:
#this last one is especially important since it will be repeated a million times each period
@btime λj * s_respond(t, hkt, lkt, lmt) + (1-λj) * s_propose(t, hkt, lkt, lmt)[1]

####  Verifying the type of the functions

Aparentemente Julia não está entendendo os tipos de argumentos das funções... 

Vejo 2 caminhos:
1. procurar como fazer para s_propose "ler" direito o tipo de output de propose
2. criar funções duplicadas, uma para s e outra para j.

Vou tentar a primeira abordagem nos próximos dias

In [ ]:
@code_warntype s_propose(t, hkt, lkt, lmt)

In [ ]:
@code_warntype threshold_j(t, lkt, lmt)

In [ ]:
@code_warntype s_respond(t, hkt, lkt, lmt)

In [ ]:
@code_warntype s_respond_liq(t, hkt, lkt, lmt)

In [ ]:
@code_warntype s_respond_payment(t, hkt, lkt, lmt)

In [ ]:
@code_warntype Pst(T,1,1)

#### Populating the penultimate period of the game

Indexed by T

In [ ]:
function populate_pkt!(t)
    
    @inbounds for l in 1:grid
        for h in 1:grid
            Pst(t, h, l)
            Pjt(t, h, l)
        end
    end

end

In [ ]:
@btime populate_pkt!(T)

In [ ]:
function populate_wkt!(t)
    
    for lm in 1:grid
        for lk in 1:grid
           for hk in lk:grid #populates only if hk >= lk, saving time
                s_W[t, hk, lk, lm] = λj * s_respond(t, hk, lk, lm) + (1-λj) * s_propose(t, hk, lk, lm)[1]
                j_W[t, hk, lk, lm] = λj * j_propose(t, hk, lk, lm)[1] + (1-λj) * j_respond(t, hk, lk, lm)
            end
        end
    end
end

In [ ]:
@btime populate_wkt!(T)

In [ ]:
@btime populate_wkt!(T, s_W, j_W)

In [ ]:
function populate_periods!(tfinal, tbegin)
    
    @time begin


        for t in tfinal:-1:tbegin

            @time begin
                #populating Pkt
                for l in 1:grid
                    for h in 1:grid
                        Pst(t, h, l)
                        Pjt(t, h, l)
                    end
                end

                #populating Wkt
                for lm in 1:grid
                    for lk in 1:grid
                        for hk in lk:grid #populates only when hk>=lk, to save time
                            s_W[t, hk, lk, lm] = λj * s_respond(t, hk, lk, lm) + (1-λj) * s_propose(t, hk, lk, lm)[1]
                            j_W[t, hk, lk, lm] = λj * j_propose(t, hk, lk, lm)[1] + (1-λj) * j_respond(t, hk, lk, lm)
                        end
                    end
                end
            end

        end
        
    end
        
end

### Populating the whole game

In [ ]:
populate_periods!(T,1)

Time necessary to populate the whole game

08/01/21: 32 minutes

12/01/21:  6.89 minutes (version 5 of respond_payment)

13/01/21: 4.081 minutes (version 5 of respond_payment (fixed) and optimized version of Pkt)

26/01/21: 5 minutes and 40 seconds (grid_payment=1000)

26/01/21: 4min15s (loop populating only if hk>=lk)

In [ ]:
257/60

### Graphs

In [ ]:
using Gadfly

In [ ]:
function graf1(t, lkt1, lkt2, lmt1, lmt2, f)
    
    set_default_plot_size(16cm, 10cm)

    #let's cut the X axis to show only values where hkt >=  max(lkt1, lkt2)
    cut = max(lkt1,lkt2)
    X = hvals[cut:grid]

    #arrays to store the outputs
    y1 = zeros(length(hvals[cut:end]))
    y2 = similar(y1)


    #by default, yi will get lkti and lmti

    #condition to avoid getting the wrong value when the function returns a tuple (ie, k_propose)
    if(f(t, 1, lkt1, lmt1)==Float64)

        for (i, hkt) in enumerate(cut:grid)
            y1[i] = f(t, hkt, lkt1, lmt1)
        end

        for (i, hkt) in enumerate(cut:grid)
            y2[i] = f(t, hkt, lkt2, lmt2)
        end
    else
        for (i, hkt) in enumerate(cut:grid)
            y1[i] = f(t, hkt, lkt1, lmt1)[1]
        end

        for (i, hkt) in enumerate(cut:grid)
            y2[i] = f(t, hkt, lkt2, lmt2)[1]
        end
    end

    
    #plotting
    
    color_y1 = "#1f77b4"
    color_y2 = "#ff7f0e"
    
    
#     possible line_styles=[:solid :dash :dot :dashdot :dashdotdot]
    p = plot()
    
    push!(p, layer(x=X, y=y1, Geom.line,
            Theme(default_color=color(color_y1),
                line_style=[:dashdotdot],
                line_width=0.4mm)))
    
    
    push!(p, layer(x=X, y=y2, Geom.line, 
            Theme(default_color=color(color_y2),
                line_style=[:dash],
                line_width=0.4mm)))
    
    


    push!(p, Guide.manual_color_key("Lower bounds",
            ["ℓst = $(lkt1/100), ℓjt = $(lmt1/100)","ℓst = $(lkt2/100), ℓjt = $(lmt2/100)"], 
            [color_y1, color_y2]))
#     push!(p, Guide.colorkey(title="Lower bounds", labels=["ℓkt = $lkt1, ℓmt = $lmt1", "ℓkt = $lkt2, ℓmt = $lmt2"]))
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Expected Payoff"))
    push!(p, Guide.title("Expected payoff of function $f"))
    
    ticksx = collect(cut/100:0.05:hhigh)
    push!(p, Guide.xticks(ticks=ticksx))

    

end





s_propose

In [ ]:
#variando lkt
graf1(1, 50, 90, 50, 50, s_propose)

In [ ]:
#variando lmt
graf1(1, 50, 50, 50, 90, s_propose)

s_respond

In [ ]:
#variando lkt
graf1(1, 30, 70, 80, 80, s_respond)

In [ ]:
#variando lmt
graf1(1, 50, 50, 50, 90, s_respond)

In [ ]:
function graf_func_value(t, lkt, lmt)
    
    
    set_default_plot_size(16cm, 10cm)
    
    #now the cut is just lkt
    X=hvals[lkt:end]
    
    y1=s_W[t, lkt:end, lkt, lmt]
    y2=j_W[t, lkt:end, lkt, lmt]
    
    
    #plotting
    
    color_y1 = "#1f77b4"
    color_y2 = "#ff7f0e"
    
    
#     possible line_styles=[:solid :dash :dot :dashdot :dashdotdot]
    p = plot()
    
    push!(p, layer(x=X, y=y1, Geom.line,
            Theme(default_color=color(color_y1),
                line_style=[:dash],
                line_width=0.4mm)))
    
    
    push!(p, layer(x=X, y=y2, Geom.line, 
            Theme(default_color=color(color_y2),
                line_style=[:dot],
                line_width=0.4mm)))
    
    


    push!(p, Guide.manual_color_key("Creditor",
            ["S","J"], 
            [color_y1, color_y2]))
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Expected Payoff"))
    push!(p, Guide.title("Continuation values at t=$(t-1), given ℓst=$(lkt/100) and ℓst=$(lmt/100)"))
    
    ticksx = collect(lkt/100:0.05:hhigh)
    push!(p, Guide.xticks(ticks=ticksx, orientation=:vertical))
    
    
    

    
  
    
    
end


graf_func_value(1,1,1)

Proposals

In [ ]:

function graf_proposal(f, t)
    
    set_default_plot_size(16cm, 10cm)
    
    #creates a dataframe to store the values
    df = DataFrame()
    df.hk = repeat(1:grid, inner=grid)
    df.lm = repeat(1:grid, outer=grid)
    
    Policy= Vector{Int64}(undef, size(df,1))
    
    for i in 1:size(df,1)
        Policy[i] = f(t, df.hk[i], 1, df.lm[i])[2]
    end
    
    df.Policy = Policy
    
#     labeldict = Dict(1=>"liq", 2=>"wait", 3=>"reorg")
    
    
#     p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point, 
#         Scale.color_discrete(levels=[1, 2, 3]))  
    
    #mudanças de cores, ficou meio jamaica
#     p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point,
#         Scale.color_discrete_manual("red","yellow", "green", levels=[1.0, 2.0, 3.0]))
    
    
    #para ficar comparável ao artigo
    p = plot(df, x=:hk, y=:lm, color=:Policy, Geom.point, alpha=[0.7],
        Scale.color_discrete_manual("black","red", "lightskyblue3", levels=[1.0, 2.0, 3.0]))
    
    push!(p, Guide.xlabel("θt"))
    push!(p, Guide.ylabel("Opponent's lower bound"))
    push!(p, Guide.title("Optimal policy, $f, at t=$(t-1)"))
    push!(p, Guide.colorkey(title="Optimal Policy", labels=["Liquidate", "Wait", "Reorganize"]))
    
                
                
end
            
#ficou muito pesado
            

In [ ]:
graf_proposal(s_propose, 1)

In [ ]:
graf_proposal(s_propose, 3)

In [ ]:
graf_proposal(j_propose,1)

In [ ]:
graf_proposal(j_propose, 3)

Analisando os valores estranhos no gráfico j_propose em t==3

In [ ]:
# show(stdout, "text/plain", Pjt_array[3, 40, 20:60, :])


In [ ]:
#para checar os valores de continuação do adversário
# s_W[4, 42, 42, 40]

In [ ]:
# show(stdout, "text/plain",U[3, 40] .*share)